In [2]:
import os
import sys
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)

import time
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior() 
import numpy as np
import models.graph as mg
import models.adversarialNets as ma
import scipy.sparse
from utils import data_process, sparse
from utils import configs, metrics, feat2struct

In [3]:
# Set random seed
seed = 123
np.random.seed(seed)
tf.set_random_seed(seed)

train_ratio = 120
batch_size = 800

namda = 0.4
beta = 0.8
print('batch_size:',batch_size)
dataset = configs.FILES.citeseer
lrate_gcn = configs.FILES.learning_rate
print("train_ratio:",train_ratio)

x, _, adj_norm, labels, train_indexes, test_indexes = data_process.load_data(dataset, str(train_ratio), 
                                                                                x_flag='feature')

node_num = adj_norm.shape[0]
label_num = labels.shape[1]

adj_norm_tuple = sparse.sparse_to_tuple(scipy.sparse.coo_matrix(adj_norm))
feat_x_nn_tuple = sparse.sparse_to_tuple(scipy.sparse.coo_matrix(x))

# node-node network train and validate masks
nn_train_mask = np.zeros([node_num,])
nn_test_mask = np.zeros([node_num,])

# batch training indexes for gan
gan_idx = tf.placeholder(tf.int32, shape=(batch_size,))
real_sample_x = tf.placeholder(tf.float32, shape=[None, label_num])

for i in train_indexes:
    nn_train_mask[i] = 1
#     nn_test_mask[i] = 0
    
for i in test_indexes:
    nn_test_mask[i] = 1
    
# TensorFlow placeholders
ph = {
      'adj_norm': tf.sparse_placeholder(tf.float32, name="adj_norm"),
      'x': tf.sparse_placeholder(tf.float32, name="features"),
      'labels': tf.placeholder(tf.float32, name="node_labels"),
      'mask': tf.placeholder(tf.int32, shape=(node_num,))
      }

placeholders = {
                'dropout_prob': tf.placeholder(tf.float32),
                'num_features_nonzero': tf.placeholder(tf.int32)
                }


# the first layer
edge_model = feat2struct.BuildGraphStruct(out_size = 30,
                                        holders=placeholders,
                                        name='e1',
                                        act = tf.nn.relu,
                                        feat_dropout = True)
edge_w = edge_model(X = x.toarray())

t_model = mg.GraphConvLayer(input_dim=x.shape[-1],
                           output_dim=configs.FILES.h,
                           name='nn_fc1',
                           holders=placeholders,
                           act=tf.nn.relu,
                           dropout=True)
  
nn_fc1, embeds_c= t_model(adj_norm_c=[ph['adj_norm'],edge_w],
                           x=ph['x'], x_c=ph['x'], sparse=True)

# nn_fc1 = 0.8*nn_fc1 + 0.2*embeds_c

# the second layer
nn_dl, embeds_c2= mg.GraphConvLayer(input_dim=configs.FILES.h,
                           output_dim=label_num,
                           name='nn_dl',
                           holders=placeholders,
                           act=tf.nn.softmax,
                           dropout=True)(adj_norm_c=[ph['adj_norm'],edge_w],
                                           x=nn_fc1, x_c=embeds_c)
                           
# nn_dl = 0.8*nn_dl + 0.2*embeds_c2

# the discriminative network
real_x = tf.gather(embeds_c2, gan_idx)
fake_x = tf.gather(nn_dl, gan_idx)

gan_model = ma.Discriminator(x_dim=label_num, h_dim=configs.FILES.hidden_dim)
D_real, D_logit_real = gan_model(real_sample_x)
D_fake, D_logit_fake = gan_model(fake_x)


def frobenius_distance(embeds_c2, original_x, edge_w, original_adj):
    original_x=tf.nn.softmax(original_x)
    embeds_c2=tf.nn.softmax(embeds_c2)
    loss =tf.norm(tf.matmul(original_x,tf.transpose(original_x))-tf.matmul(embeds_c2,tf.transpose(embeds_c2)),
                       ord='fro', axis=(0,1))
    return loss

def gan_sigmoid_cross_entropy(D_real_preds, D_fake_preds):
    
    D_loss_real = tf.nn.sigmoid_cross_entropy_with_logits(logits=D_real_preds, 
                                                          labels=tf.ones_like(D_real_preds))
    D_loss_fake = tf.nn.sigmoid_cross_entropy_with_logits(logits=D_fake_preds, 
                                                          labels=tf.zeros_like(D_fake_preds))
    loss_g = tf.nn.sigmoid_cross_entropy_with_logits(logits=D_fake_preds, 
                                                     labels=tf.ones_like(D_fake_preds))
    
    loss_d = tf.reduce_mean(D_loss_real) + tf.reduce_mean(D_loss_fake)
    loss_g = tf.reduce_mean(loss_g)
#     return tf.reduce_mean(D_loss_real) + tf.reduce_mean(D_loss_fake) + tf.reduce_mean(loss_g)
    return loss_d, loss_g

def masked_sigmoid_softmax_cross_entropy(preds, labels, mask):
    """Sigmoid softmax cross-entropy loss with masking."""
    loss = tf.nn.softmax_cross_entropy_with_logits(logits=preds, labels=labels)
    mask = tf.cast(mask, dtype=tf.float32)
    mask /= tf.reduce_mean(mask)
    loss *= mask
#     for var in tf.trainable_variables():
    for var in t_model.var.values():
        var = tf.cast(var, dtype=tf.float32)
        loss += configs.FILES.weight_decay * tf.nn.l2_loss(var)
    return tf.reduce_mean(loss)

def masked_accuracy(preds, labels, mask):
    """Accuracy with masking."""
    correct_prediction = tf.equal(tf.argmax(preds, 1), tf.argmax(labels, 1))
    accuracy_all = tf.cast(correct_prediction, tf.float32)
    mask = tf.cast(mask, dtype=tf.float32)
    mask /= tf.reduce_mean(mask)
    accuracy_all *= mask
    
    return tf.reduce_mean(accuracy_all), correct_prediction

# calculate the classification accuracy per classes   
def precision_per_class(preds, labels, mask):
    import heapq
    mask = mask.astype(int)
    labels = labels.astype(int)
    val_indexes = np.where(mask==1)[0]
    pred_true_labels = {}
    
    y_true = []
    y_pred = []    
    
    for i in val_indexes:
        pred_probs_i = preds[i]
        true_raw_i = labels[i]
        
        pred_label_i = heapq.nlargest(np.sum(true_raw_i),range(len(pred_probs_i)), 
                                      pred_probs_i.take)
        true_label_i = np.where(true_raw_i==1)[0]
        pred_true_labels[i] = (pred_label_i, true_label_i)
        
        y_true.append(true_label_i)
        y_pred.append(pred_label_i)
        
    accuracy_per_classes = metrics.evaluate(pred_true_labels)
    
    from sklearn.metrics import roc_curve, auc, confusion_matrix
    
    mat = confusion_matrix(y_true, y_pred)
    print(mat)        
        
    fpr = dict()
    tpr = dict()
    test_y = labels[val_indexes]
    test_pred = preds[val_indexes]
    fpr["micro"], tpr["micro"], _ = roc_curve(test_y.ravel(), test_pred.ravel())
    auc = auc(fpr["micro"], tpr["micro"])
    
    return accuracy_per_classes, auc
    
    from sklearn.metrics import roc_curve, auc
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    test_y = labels[val_indexes]
    test_pred = preds[val_indexes]
    fpr["micro"], tpr["micro"], _ = roc_curve(test_y.ravel(), test_pred.ravel())
    roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])
    print('micro_auc=',roc_auc["micro"])
    
    return accuracy_per_classes

with tf.name_scope('optimizer'):
    # graph construct loss L_{cont}
    graph_build_loss = frobenius_distance(embeds_c2,
                                          tf.dtypes.cast(x.toarray(), tf.float32),
                                          tf.sparse.to_dense(edge_w),
                                          tf.dtypes.cast(adj_norm.toarray(), tf.float32))
    
    # discriminator training loss L_{gan}
    d_loss, g_loss = gan_sigmoid_cross_entropy(D_real_preds=D_logit_real, 
                                               D_fake_preds=D_logit_fake)
    
    # semi-supervised node classification loss L_{gcn}
    class_loss = masked_sigmoid_softmax_cross_entropy(preds=nn_dl, 
                                                labels=ph['labels'], 
                                                mask=ph['mask'])
#     
    loss1 = namda*graph_build_loss + 0.8*d_loss + class_loss
    loss2 = namda*graph_build_loss + 0.8*g_loss + class_loss
        
    accuracy, correct_prediction = masked_accuracy(preds=nn_dl, 
                               labels=ph['labels'], mask=ph['mask'])
    
    optimizer = tf.train.AdamOptimizer(learning_rate=lrate_gcn)
#     opt_op = optimizer.minimize(loss)    
    opt_op1 = optimizer.minimize(loss1)    
    opt_op2 = optimizer.minimize(loss2)    

feed_dict_train = {ph['adj_norm']: adj_norm_tuple,
                      ph['x']: feat_x_nn_tuple,
#                       ph['labels']: labels,
                      ph['labels']: labels.toarray(),
                      ph['mask']: nn_train_mask,
                      placeholders['dropout_prob']: configs.FILES.dropout_prob,
                      placeholders['num_features_nonzero']: feat_x_nn_tuple[1].shape,
                      gan_idx: None,
                      real_sample_x: None
                      }
feed_dict_val = {ph['adj_norm']: adj_norm_tuple,
                    ph['x']: feat_x_nn_tuple,
#                     ph['labels']: labels,
                    ph['labels']: labels.toarray(),
                    ph['mask']: nn_test_mask,
                    placeholders['dropout_prob']: 0.,
                    placeholders['num_features_nonzero']: feat_x_nn_tuple[1].shape,
                    gan_idx: None
                    }

sess = tf.Session()
sess.run(tf.global_variables_initializer())

epochs = 400
save_every = 1    
    
t = time.time()
# Train model
times = []

for epoch in range(epochs):
    # Training node embedding
#     _, train_loss = sess.run(
#         (opt_op, loss), feed_dict=feed_dict_train)
    # obtain the training sample indexes for gan
    begin = epoch * batch_size
    end = epoch * batch_size + batch_size
    batch_idx = []
    for i in range(begin, end):
        idx = i % node_num
        batch_idx.append(idx)

    if epoch % save_every == 0:
        feed_dict_val.update(({gan_idx: batch_idx}))
        val_acc, test_nn_dl, real_x_v = sess.run((accuracy, nn_dl, real_x), feed_dict=feed_dict_val)
    feed_dict_train.update(({gan_idx: batch_idx, real_sample_x: real_x_v}))
    sess.run((opt_op1, graph_build_loss, accuracy, nn_dl),feed_dict=feed_dict_train) 
    _, dd_loss, gg_loss, cclass_loss, train_acc, train_nn_dl = sess.run((opt_op2, d_loss, g_loss, class_loss, accuracy, nn_dl), 
                                                              feed_dict=feed_dict_train) 
    
    print("Epoch:", '%04d' % (epoch + 1),
          "dd_loss=", "{:.5f}".format(dd_loss),
          "gg_loss=", "{:.5f}".format(gg_loss),
          "cclass_loss=", "{:.5f}".format(cclass_loss),
          "train_acc=", "{:.5f}".format(train_acc),
#               "train_auc", "{:.5f}".format(train_auc),
          "test_acc=", "{:.5f}".format(val_acc),
#               "test_auc", "{:.5f}".format(test_auc),
          "time=", "{:.5f}".format(time.time() - t))

    if epoch % 20 == 0:
        times.append(time.time() - t)

batch_size: 800
train_ratio: 120
label_distribution: [('0', 20), ('1', 20), ('2', 20), ('3', 20), ('4', 20), ('5', 20)]
balance_num: 20
(3312, 3703) (3312, 3312) (3312, 6) (120,) (1000,)
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use `tf.keras.layers.Conv1D` instead.
Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



/home/mins/.local/lib/python3.8/site-packages/tensorflow/python/framework/indexed_slices.py:432: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(


Epoch: 0001 dd_loss= 1.38209 gg_loss= 0.69872 cclass_loss= 1.80074 train_acc= 0.25000 test_acc= 0.12400 time= 14.95932
Epoch: 0002 dd_loss= 1.38677 gg_loss= 0.67778 cclass_loss= 1.79252 train_acc= 0.35000 test_acc= 0.27200 time= 22.86595
Epoch: 0003 dd_loss= 1.38750 gg_loss= 0.65962 cclass_loss= 1.78169 train_acc= 0.52500 test_acc= 0.31200 time= 30.01443
Epoch: 0004 dd_loss= 1.38971 gg_loss= 0.64379 cclass_loss= 1.76973 train_acc= 0.64167 test_acc= 0.37400 time= 36.91939
Epoch: 0005 dd_loss= 1.39048 gg_loss= 0.63052 cclass_loss= 1.75416 train_acc= 0.75000 test_acc= 0.42600 time= 43.94264
Epoch: 0006 dd_loss= 1.39336 gg_loss= 0.61664 cclass_loss= 1.73740 train_acc= 0.85833 test_acc= 0.46400 time= 51.79827
Epoch: 0007 dd_loss= 1.39673 gg_loss= 0.60323 cclass_loss= 1.71679 train_acc= 0.84167 test_acc= 0.51800 time= 60.35088
Epoch: 0008 dd_loss= 1.39937 gg_loss= 0.59114 cclass_loss= 1.70336 train_acc= 0.85833 test_acc= 0.53800 time= 68.29365
Epoch: 0009 dd_loss= 1.39781 gg_loss= 0.58300 cc

Epoch: 0070 dd_loss= 1.36383 gg_loss= 0.45460 cclass_loss= 1.17094 train_acc= 0.99167 test_acc= 0.70200 time= 510.04650
Epoch: 0071 dd_loss= 1.32032 gg_loss= 0.48423 cclass_loss= 1.16209 train_acc= 0.99167 test_acc= 0.70700 time= 516.18740
Epoch: 0072 dd_loss= 1.31303 gg_loss= 0.48639 cclass_loss= 1.15002 train_acc= 1.00000 test_acc= 0.70400 time= 522.69266
Epoch: 0073 dd_loss= 1.33441 gg_loss= 0.47174 cclass_loss= 1.15465 train_acc= 0.99167 test_acc= 0.70300 time= 529.20420
Epoch: 0074 dd_loss= 1.35288 gg_loss= 0.45986 cclass_loss= 1.15373 train_acc= 0.99167 test_acc= 0.70000 time= 535.77229
Epoch: 0075 dd_loss= 1.32614 gg_loss= 0.47575 cclass_loss= 1.16237 train_acc= 0.98333 test_acc= 0.70500 time= 542.01070
Epoch: 0076 dd_loss= 1.29364 gg_loss= 0.49677 cclass_loss= 1.15993 train_acc= 0.99167 test_acc= 0.70800 time= 548.18898
Epoch: 0077 dd_loss= 1.32602 gg_loss= 0.47334 cclass_loss= 1.16017 train_acc= 0.98333 test_acc= 0.70500 time= 554.34701
Epoch: 0078 dd_loss= 1.33513 gg_loss= 0.

Epoch: 0139 dd_loss= 1.21701 gg_loss= 0.51345 cclass_loss= 1.14436 train_acc= 0.99167 test_acc= 0.71100 time= 851.45859
Epoch: 0140 dd_loss= 1.21701 gg_loss= 0.51212 cclass_loss= 1.14134 train_acc= 0.99167 test_acc= 0.70800 time= 857.30309
Epoch: 0141 dd_loss= 1.21948 gg_loss= 0.51255 cclass_loss= 1.13948 train_acc= 0.99167 test_acc= 0.70900 time= 865.80336
Epoch: 0142 dd_loss= 1.15919 gg_loss= 0.55241 cclass_loss= 1.13020 train_acc= 0.99167 test_acc= 0.70800 time= 875.72193
Epoch: 0143 dd_loss= 1.20283 gg_loss= 0.51895 cclass_loss= 1.14044 train_acc= 1.00000 test_acc= 0.70900 time= 885.65713
Epoch: 0144 dd_loss= 1.21033 gg_loss= 0.51496 cclass_loss= 1.13904 train_acc= 1.00000 test_acc= 0.71500 time= 895.12109
Epoch: 0145 dd_loss= 1.23345 gg_loss= 0.50144 cclass_loss= 1.12786 train_acc= 1.00000 test_acc= 0.71000 time= 903.29710
Epoch: 0146 dd_loss= 1.16243 gg_loss= 0.55365 cclass_loss= 1.13608 train_acc= 1.00000 test_acc= 0.70700 time= 916.70202
Epoch: 0147 dd_loss= 1.19231 gg_loss= 0.

Epoch: 0207 dd_loss= 1.16380 gg_loss= 0.52419 cclass_loss= 1.13105 train_acc= 1.00000 test_acc= 0.71600 time= 1416.21944
Epoch: 0208 dd_loss= 1.06704 gg_loss= 0.58979 cclass_loss= 1.13152 train_acc= 0.99167 test_acc= 0.71500 time= 1424.13856
Epoch: 0209 dd_loss= 1.13085 gg_loss= 0.55298 cclass_loss= 1.13546 train_acc= 0.99167 test_acc= 0.71300 time= 1432.78804
Epoch: 0210 dd_loss= 1.12629 gg_loss= 0.55301 cclass_loss= 1.12626 train_acc= 0.99167 test_acc= 0.71100 time= 1445.06141
Epoch: 0211 dd_loss= 1.16992 gg_loss= 0.52625 cclass_loss= 1.13038 train_acc= 1.00000 test_acc= 0.71500 time= 1454.82662
Epoch: 0212 dd_loss= 1.09207 gg_loss= 0.57375 cclass_loss= 1.13004 train_acc= 0.99167 test_acc= 0.71400 time= 1463.25834
Epoch: 0213 dd_loss= 1.09225 gg_loss= 0.57311 cclass_loss= 1.13932 train_acc= 0.99167 test_acc= 0.71400 time= 1470.53698
Epoch: 0214 dd_loss= 1.12300 gg_loss= 0.55369 cclass_loss= 1.13537 train_acc= 0.99167 test_acc= 0.71500 time= 1477.08079
Epoch: 0215 dd_loss= 1.15873 gg_

Epoch: 0275 dd_loss= 1.01682 gg_loss= 0.60707 cclass_loss= 1.12527 train_acc= 0.99167 test_acc= 0.72000 time= 1925.21103
Epoch: 0276 dd_loss= 1.03591 gg_loss= 0.58914 cclass_loss= 1.12283 train_acc= 1.00000 test_acc= 0.72100 time= 1931.99525
Epoch: 0277 dd_loss= 1.08689 gg_loss= 0.55915 cclass_loss= 1.12646 train_acc= 0.99167 test_acc= 0.72100 time= 1938.67093
Epoch: 0278 dd_loss= 1.03075 gg_loss= 0.59406 cclass_loss= 1.13929 train_acc= 0.99167 test_acc= 0.72200 time= 1945.05637
Epoch: 0279 dd_loss= 1.01863 gg_loss= 0.60419 cclass_loss= 1.12234 train_acc= 0.99167 test_acc= 0.72300 time= 1951.67533
Epoch: 0280 dd_loss= 1.01959 gg_loss= 0.60311 cclass_loss= 1.13085 train_acc= 0.99167 test_acc= 0.72100 time= 1958.86930
Epoch: 0281 dd_loss= 1.08053 gg_loss= 0.56480 cclass_loss= 1.13007 train_acc= 1.00000 test_acc= 0.71900 time= 1965.52296
Epoch: 0282 dd_loss= 1.06391 gg_loss= 0.57128 cclass_loss= 1.13601 train_acc= 1.00000 test_acc= 0.72100 time= 1972.79617
Epoch: 0283 dd_loss= 1.00831 gg_

Epoch: 0343 dd_loss= 1.03640 gg_loss= 0.58445 cclass_loss= 1.12731 train_acc= 1.00000 test_acc= 0.72400 time= 2421.59794
Epoch: 0344 dd_loss= 1.02070 gg_loss= 0.59388 cclass_loss= 1.12784 train_acc= 0.99167 test_acc= 0.71800 time= 2429.14601
Epoch: 0345 dd_loss= 0.96874 gg_loss= 0.63626 cclass_loss= 1.13432 train_acc= 0.99167 test_acc= 0.71300 time= 2436.18298
Epoch: 0346 dd_loss= 0.99221 gg_loss= 0.61216 cclass_loss= 1.13346 train_acc= 1.00000 test_acc= 0.71700 time= 2442.98969
Epoch: 0347 dd_loss= 1.00567 gg_loss= 0.60541 cclass_loss= 1.12476 train_acc= 1.00000 test_acc= 0.72400 time= 2451.26430
Epoch: 0348 dd_loss= 1.02093 gg_loss= 0.59317 cclass_loss= 1.11702 train_acc= 0.99167 test_acc= 0.72200 time= 2461.33910
Epoch: 0349 dd_loss= 0.95130 gg_loss= 0.64684 cclass_loss= 1.13432 train_acc= 0.99167 test_acc= 0.72000 time= 2470.22738
Epoch: 0350 dd_loss= 0.99600 gg_loss= 0.61342 cclass_loss= 1.13516 train_acc= 0.99167 test_acc= 0.72000 time= 2476.61410
Epoch: 0351 dd_loss= 1.00654 gg_